In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("alexistubulekasA3_partA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.cores.max",2)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

In [3]:
# A.1.1 Read the English transcripts with Spark, and count the number of lines.

rdd_en = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': '\n'}
)\
.cache() # Keep this RDD in memory!

rdd_en.count()

1862234

In [4]:
# A.1.2 Do the same with the other language (so that you have a separate lineage of RDDs for each).
rdd_sv = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': '\n'}
)\
.cache() # Keep this RDD in memory!
rdd_sv.count()

1862234

In [ ]:
# A.1.3 Verify that the line counts are the same for the two languages.

# Ans: Yes they are the same as seen above

In [5]:
# A.1.4 Count the number of partitions.
print(rdd_en.getNumPartitions())
print(rdd_sv.getNumPartitions())

2
3


In [6]:
#A2.1 Pre-process the text from both RDDs by doing the following: Lowercase the text + Tokenize the text (split on space)

def lowercase_and_split(rdd_input):
    """Takes in a rdd and outputs all the values in second position in the tuple
    lowercased and splited on space"""
    return rdd_input.map(lambda w: w[1].lower().split(' '))

# A.2.2 Inspect 10 entries from each of your RDDs to verify your pre-processing.
rdd_en_lower_split = lowercase_and_split(rdd_en)
rdd_sv_lower_split = lowercase_and_split(rdd_sv)

print(rdd_en_lower_split.take(10)) #print 10 entries from English text
print(rdd_sv_lower_split.take(10)) #print 10 entries from Swedish text

# A.2.3 Verify that the line counts still match after the pre-processing
print(rdd_en_lower_split.count())
print(rdd_sv_lower_split.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

In [7]:
# A.3.1 Use Spark to compute the 10 most frequently according words in the English language corpus. Repeat for the other language.
from operator import add

#English text
all_words_en = rdd_en_lower_split.flatMap(lambda w: w)\
    .map(lambda w: (w,1))

word_counts_en = all_words_en.reduceByKey(add)
print(word_counts_en.takeOrdered(10, key=lambda x: -x[1]))

#Swedish text
all_words_sv = rdd_sv_lower_split.flatMap(lambda w: w)\
    .map(lambda w: (w,1))

word_counts_sv = all_words_sv.reduceByKey(add)
print(word_counts_sv.takeOrdered(10, key=lambda x: -x[1]))

[('the', 3498375), ('of', 1659758), ('to', 1539760), ('and', 1288401), ('in', 1085993), ('that', 797516), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522849)]
[('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924866), ('som', 913276), ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]


In [ ]:
# A.3.2 Verify that your results are reasonable.

# Ans: Yes, the results are reasonable. If we look above at the top 10 most common words in both languanges we see that they are 
# very similar to each other and they are also what you would expect - a lot of prepositions. 

In [8]:
#A.4.1
#1. Key the lines by their line number
en_1 = rdd_en_lower_split
en_1 = en_1.zipWithIndex()

sv_1 = rdd_sv_lower_split
sv_1 = sv_1.zipWithIndex()

In [9]:
# 2. Swap the key and value - so that the line number is the key
en_2 = en_1.map(lambda w: (w[1],w[0]))
sv_2 = sv_1.map(lambda w: (w[1],w[0]))

In [10]:
# 3. Join on key
joined_en_sv_3 = en_2.join(sv_2)

In [11]:
# 4. Filter to exclude line pairs that have an empty/missing “corresponding” sentence.
joined_en_sv_4 = joined_en_sv_3.filter(lambda w:w[1][0]!=[''])\
                   .filter(lambda w:w[1][1]!=[''])


In [12]:
# 5. Filter to have only pairs of sentences with a small number of words per sentence (I use 4 words or less)
joined_en_sv_5 = joined_en_sv_4.filter(lambda w:len(w[1][0])<5)\
                               .filter(lambda w:len(w[1][1])<5)



In [13]:
# 6. Filter to leave only pairs of sentences with the same number of words in each sentence
joined_en_sv_6 = joined_en_sv_5.filter(lambda w:len(w[1][0])==len(w[1][1]))
                               



In [14]:
# 7 For each sentence pair, map so that you pair each (in order) word in the two sentences. We no longer need the line numbers. (hint: use python’s built in zip() function)
#Step 1: drop line number
#Step 2: zip corresponding words
#Step 3: #return as tuple

joined_en_sv_7 = joined_en_sv_6.map(lambda w:w[1])\
                               .map(lambda w:zip(w[0],w[1]))\
                               .map(lambda w:tuple(w))
 

In [15]:
# 8. Use reduce to count the number of occurrences of the word-translation-pairs
from operator import add

joined_en_sv_8 = joined_en_sv_7.flatMap(lambda w: (w))\
                               .map(lambda w: (w,1))

word_counts_joined_en_sv_8 = joined_en_sv_8.reduceByKey(add)
print(word_counts_joined_en_sv_8.takeOrdered(30, key=lambda x: -x[1])) #print some of the most frequent occuring pairs of words 

[(('(applause)', '(applåder)'), 2546), (('closed.', 'avslutad.'), 2534), (('is', 'är'), 2380), (('.', '.'), 2082), (('is', 'debatten'), 1324), (('the', 'jag'), 1324), (('debate', 'förklarar'), 1317), (('the', 'debatten'), 1225), (('is', 'härmed'), 1215), (('debate', 'är'), 1187), (('(rule', '(artikel'), 893), (('that', 'det'), 852), (('written', 'skriftliga'), 847), (('\xa0\xa0', '\xa0\xa0'), 842), (('statements', 'förklaringar'), 801), (('we', 'vi'), 636), (('i', 'jag'), 629), (('this', 'detta'), 582), (('142)', '142)'), 557), (('it', 'det'), 515), (('applause', 'applåder'), 461), (('1.', '1.'), 438), (('2.', '2.'), 438), (('there', 'det'), 429), (('3.', '3.'), 405), (('why?', 'varför?'), 372), (('-', '-'), 367), (('are', 'är'), 364), (('this', 'det'), 361), (('are', 'finns'), 360)]


In [57]:
# Q: Do the translations seem resonable?

#Answer:  Most of them makes sense. "Applause", "closed", "is", "rule" and "written" are some of the words that get correct translations. Some are wrong like ('is','debatten'). 
# Most likely due to the fact that in English the definite article (the) is used before a noun to indicate that the identity of the noun is known to the reader, while in Swedish 
# this is done by changing the ending of the word itself. 
# There are a few number characters and other wierd characters that one would preferrably filter out. 

In [71]:
# release the cores for another application!
spark_context.stop()